In [94]:
import math
import nltk
from nltk import word_tokenize,sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rajarshidas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# PREPROCESSING

In [148]:
text=""
for i in range (1,2):
    filename='Harry_Potter_Text/'+'Book'+str(i)+'.txt'
    with open(filename) as f:
        Lines = f.readlines()
        # book=""
        for lin in Lines:
            if not (len(lin)>5 and lin[:6]=='Page |'):
                if lin != 'Harry Potter and the Goblet of Fire - J.K. Rowling \n':
                    text+=lin.lower()
x=sent_tokenize(text)
sentences=[]
sentencewise_tokens=[]
for l in x:
    li=''.join(l.splitlines())
    # if not li[-1].isalnum():
    #     li=li[:-1]
    # li= '<s> '+li+' <e>'
    sentences.append(li)
    sentencewise_tokens.append(word_tokenize(li))
tokens=word_tokenize(text)



In [149]:
len(sentences)

5030

In [150]:
# sentences

In [151]:
len(tokens)

101222

In [152]:
len(sentencewise_tokens)

5030

In [153]:
final_tokens=[]
for token in tokens:
    if not (len(token)==1 and not token.isalnum()):
        final_tokens.append(token.strip('.'))
tokens=final_tokens
#REMOVE PUNCTUATION

In [154]:
len(tokens)

80585

In [133]:
vocab=list(set(tokens))

In [134]:
print(len(vocab))

21079


# N-GRAM MODEL

In [12]:
#value of m in problem statement/ range of n in n-gram
n_limit=5

In [13]:
ngrams=[]
for i in range (n_limit):
    d={}
    ngrams.append(d)
for n in range (1,n_limit+1):
    for i in range(len(tokens)):
        if i+n>len(tokens):
            break
        lis=[]
        for j in range(i,i+n):
            lis.append(tokens[j])
        lis=tuple(lis)
        if lis not in ngrams[n-1]:
            ngrams[n-1][lis] = 1
        else : 
            ngrams[n-1][lis] += 1
    ngrams[n-1]=dict(sorted(ngrams[n-1].items(), key=lambda item: -item[1]))
    print(n,len(ngrams[n-1]))




1 21079
2 277884
3 642415
4 835723
5 896438


In [14]:
# ngrams[4]

# SENTENCE GENERATION

In [15]:
def get_n(n,a,line,word_limit):
    if word_limit==0:
        print(line)
        return line
    sz=len(a)
    if sz==0 or n==1:
        word=next(iter(ngrams[0]))[0]#first key
        a.append(word)
        line=line+' '+word
        # print(line)
        get_n(n,a,line,word_limit-1)
    elif sz<n-1:
        found = False
        for key in ngrams[sz]:
            lis=list(key)
            last_word=lis[-1]
            lis=lis[:-1]
            if lis==a:
                line=line+' '+last_word
                # a=a[1:]
                a.append(last_word)
                found=True
                break
        if found :
            # print(line)
            get_n(n,a,line,word_limit-1)
        else : 
            print('Cant find')
    elif sz==n-1:
        found = False
        for key in ngrams[n-1]:
            lis=list(key)
            last_word=lis[-1]
            lis=lis[:-1]
            if lis==a:
                line=line+' '+last_word
                a=a[1:]
                a.append(last_word)
                found=True
                break
        if found :
            # print(line)
            get_n(n,a,line,word_limit-1)
        else : 
            print('Cant find')
        




In [16]:
def generate_sentence(n,word_limit):
    prev_tokens=[]
    sentence=""
    line=get_n(n,prev_tokens,sentence,word_limit)  


In [95]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [18]:
generate_sentence(5,100)

 the door and turned the key fumbling in their panic harry pulled the door open and they ran inside hermione granger was shrinking against the wall opposite looking as if she was about to faint the troll was advancing on her knocking the sinks off the walls as it went confuse it harry said desperately to ron and seizing a tap he threw it as hard as he could into the kitchen harry hurried into the living room in time to catch the last report on the evening news and finally bird-watchers everywhere have reported that the nation s owls


# DEV SET

In [456]:
text=""
filename='Harry_Potter_Text/'+'Book7.txt'
with open(filename) as f:
    Lines = f.readlines()
    # book=""
    for lin in Lines:
        if not (len(lin)>5 and lin[:6]=='Page |'):
            if lin != 'Harry Potter and the Goblet of Fire - J.K. Rowling \n':
                text+=lin.lower()
x=sent_tokenize(text)
test_sentences=[]
test_sentencewise_tokens=[]
for l in x:
    li=''.join(l.splitlines())
    # if not li[-1].isalnum():
    #     li=li[:-1]
    # li= '<s> '+li+' <e>'
    test_sentences.append(li)
    test_sentencewise_tokens.append(word_tokenize(li))
test_tokens=word_tokenize(text)



In [457]:
len(test_sentences)

11138

In [458]:
len(test_tokens)

256058

In [459]:
test_final_tokens=[]
for token in test_tokens:
    if not (len(token)==1 and not token.isalnum()):
        test_final_tokens.append(token.strip('.'))
test_tokens=test_final_tokens
#REMOVE PUNCTUATION

In [460]:
len(test_tokens)

204766

In [461]:
test_vocab=set(test_tokens)

In [463]:
len(test_vocab)

11449

In [464]:
vocab=set(vocab)

In [466]:
#OOV words
len(test_vocab.difference(vocab))

2068

In [468]:
# test_vocab.difference(vocab)

# PERPLEXITY

In [20]:
generate_sentence(2,10)

 the door and the door and the door and the


In [21]:
len(vocab)

21079

In [22]:
def log_perp(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        # print(sz)
        if sz==1:
            tup=tuple(lis)
            ratio=ngrams[0][tup]/len(tokens)
            # print(ratio)
            sum+=math.log2(ratio)
        else:
            tup1=tuple(lis)
            lis=lis[:-1]
            tup2=tuple(lis)
            ratio=ngrams[sz-1][tup1]/ngrams[sz-2][tup2]
            sum+=math.log2(ratio)
    return sum*(-1/len(words))

In [23]:
s='the door and the door and the door and the'
print(log_perp(2,s))


4.579554625674844


# SMOOTHING

### ADD-1 

In [26]:
def log_perp1(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        # print(sz)
        if sz==1:
            tup=tuple(lis)
            if tup in ngrams[0]:
                ratio=(ngrams[0][tup]+1)/(len(tokens)+len(vocab))
            else:
                ratio=(1)/(len(tokens)+len(vocab))
            sum+=math.log2(ratio)
        else:
            tup1=tuple(lis)
            lis=lis[:-1]
            tup2=tuple(lis)
            if tup1 in ngrams[sz-1]:
                ratio=(ngrams[sz-1][tup1]+1)/(ngrams[sz-2][tup2]+len(ngrams[sz-2]))
            elif tup2 in ngrams[sz-2]:
                ratio=1/(ngrams[sz-2][tup2]+len(ngrams[sz-2]))
            else:
                #ERROR WARNING
                ratio=1/len(ngrams[sz-2])
            sum+=math.log2(ratio)
    return sum*(-1/len(words))

In [31]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [47]:
#comparison over generated sentences
s='the door and turned the key fumbling in their panic'
n_val=5
print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with add-1 smoothing is',log_perp1(n_val,s))

Log2-perplexity without smoothing is 1.9812421762636143
Log2-perplexity with add-1 smoothing is 15.077224482717945


### GOOD TURING

In [35]:
ngrams_total=[] #holds total number of n gram tuples per n
ngrams_freq=[]  #holds dictionary of frequency and count per n
for i in range (n_limit):
    d={}
    ngrams_freq.append(d)
for n in range (n_limit):
    sum=0
    for val in ngrams[n].values():
        # print(val,n)
        if val in ngrams_freq[n]:
            ngrams_freq[n][val]=ngrams_freq[n][val]+1
        else:
            ngrams_freq[n][val]=1
        sum+=val
    ngrams_total.append(sum)

In [36]:
ngrams_total

[920732, 920731, 920730, 920729, 920728]

In [124]:
# ngrams_freq[4]

In [44]:
def log_perp2(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        # print(sz)
        if sz==1:
            tup=tuple(lis)
            if tup in ngrams[0]:
                freq=ngrams[0][tup]
                if freq+1 in ngrams_freq[0]:
                    ratio=(freq+1)*ngrams_freq[0][freq+1]/(ngrams_freq[0][freq]*ngrams_total[0])
                else:
                    ratio=(freq+1)/(ngrams_freq[0][freq]*ngrams_total[0])
            else:
                if 1 in ngrams_freq[0]:
                    ratio=ngrams_freq[0][1]/len(tokens)
                else :
                    ratio=1/len(tokens)
            sum+=math.log2(ratio)
        else:
            tup1=tuple(lis)
            lis=lis[:-1]
            tup2=tuple(lis)
            if tup1 in ngrams[sz-1]:
                freq=ngrams[sz-1][tup1]
                if freq+1 in ngrams_freq[sz-1]:
                    ratio=(freq+1)*ngrams_freq[sz-1][freq+1]/(ngrams_freq[sz-1][freq]*ngrams_total[sz-1])
                else:
                    ratio=(freq+1)/(ngrams_freq[sz-1][freq]*ngrams_total[sz-1])
            else:
                if 1 in ngrams_freq[sz-1]:
                    ratio=ngrams_freq[sz-1][1]/ngrams_total[sz-1]
                else:
                    ratio=1/ngrams_total[sz-1]
            sum+=math.log2(ratio)
    return sum*(-1/len(words))

In [45]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [48]:
#comparison over generated sentences
s='the door and turned the key fumbling in their panic'
n_val=5
print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with good turing smoothing is',log_perp2(n_val,s))

Log2-perplexity without smoothing is 1.9812421762636143
Log2-perplexity with good turing smoothing is 18.961096391546445


### BACKOFF

In [88]:
def log_perp3(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        # print(sz)
        if sz==1:
            tup=tuple(lis)
            ratio=ngrams[0][tup]/len(tokens)
            sum+=math.log2(ratio)
        else:
            tup1=tuple(lis)
            if tup1 in ngrams[sz-1]:
                
                lis=lis[:-1]
                tup2=tuple(lis)
                ratio=ngrams[sz-1][tup1]/ngrams[sz-2][tup2]
                sum+=math.log2(ratio)
            else :
                lis=lis[1:]
                snew=' '.join(lis)
                back=-1*log_perp3(n-1,snew)*len(lis)
                # print(snew,back)
                sum+=back
                sum+=math.log2(0.4)
                
    return sum*(-1/len(words))

In [89]:
print(log_perp(1,'the'))
print(log_perp(1,'animagi'))
print(log_perp3(2,'animagi the'))
print(log_perp3(1,'the'))

4.4733372813656676
17.00506684057854
11.400166108415785
4.4733372813656676


In [90]:
# log_perp(2,'animagi the')

In [91]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [92]:
#comparison over generated sentences
s='the door and turned the key fumbling in their panic'
n_val=5
print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with backoff smoothing is',log_perp3(n_val,s))

Log2-perplexity without smoothing is 1.9812421762636143
Log2-perplexity with backoff smoothing is 1.9812421762636143


In [93]:
#comparison over generated sentences
s='the door and turned the key fumbling in their ministry'
n_val=5
# print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with backoff smoothing is',log_perp3(n_val,s))

Log2-perplexity with backoff smoothing is 7.710051297553377


### KNESSER-NEY

In [106]:
ngrams_unique_prefix=[]     # for each n gram model contains number of unique w_i-1 for each w_i as key
ngrams_unique_suffix=[]     # for each n gram model contains number of unique w_i for each w_i-1 as key
for i in range (n_limit):
    d={}
    ngrams_unique_prefix.append(d)
    ngrams_unique_suffix.append(d)
for n in range (1,n_limit):
    for key in ngrams[n]:
        lis=list(key)
        lastword=lis[-1]
        lis=tuple(lis[:-1])
        if lastword in ngrams_unique_prefix[n]:
            ngrams_unique_prefix[n][lastword]=ngrams_unique_prefix[n][lastword]+1
        else:
            ngrams_unique_prefix[n][lastword]=1
        if lis in ngrams_unique_suffix[n]:
            ngrams_unique_suffix[n][lis]=ngrams_unique_suffix[n][lis]+1
        else:
            ngrams_unique_suffix[n][lis]=1

In [112]:
def log_perp4(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        # Absolute discounting
        if sz==1:
            tup=tuple(lis)
            if tup in ngrams[0]:
                ratio=(ngrams[0][tup]-0.75)/len(tokens)
            else :
                ratio = 1/len(tokens)
            sum+=math.log2(ratio)
        else:
            tup1=tuple(lis)
            lis=lis[:-1]
            tup2=tuple(lis)

            # CONTINUATION PROBABILITY
            lastword=lis[-1]
            # pcont_ratio=0
            pcont_ratio=ngrams_unique_prefix[sz-1][lastword]/len(ngrams[sz-1])

            # LAMBDA
            if tup2 in ngrams_unique_suffix[sz-1]:
                lambdamult=0.75*ngrams_unique_suffix[sz-1][tup2]/ngrams[sz-2][tup2]
            else: 
                lambdamult=1
            # lambdamult=1

            if tup1 in ngrams[sz-1]:
                # add-1 smoothing
                # ratio=(max(ngrams[sz-1][tup1]-0.75,0)+1)/(ngrams[sz-2][tup2]+ngrams[sz-2])
                ratio=max(ngrams[sz-1][tup1]-0.75,0)/(ngrams[sz-2][tup2])
                sum+=math.log2(ratio+lambdamult*pcont_ratio)
            else :
                ratio=1/len(ngrams[sz-2])
                sum+=math.log2(ratio+lambdamult*pcont_ratio)

    return sum*(-1/len(words))

In [114]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [113]:
#comparison over generated sentences
s='the door and turned the key fumbling in their panic'
n_val=5
print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with KN is',log_perp4(n_val,s))

Log2-perplexity without smoothing is 1.9812421762636143
Log2-perplexity with KN is 2.88767992295894


### INTERPOLATION

In [121]:
def log_perp5(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        
        ratio_lis=[]
        while True:
            tup1=tuple(lis)
            if len(lis)==1:
                ratio=ngrams[0][tup1]/len(tokens)
                ratio_lis.append(ratio)
                break
            elif tup1 in ngrams[len(lis)-1]:
                lis2=lis[:-1]
                tup2=tuple(lis2)
                ratio=ngrams[len(lis)-1][tup1]/ngrams[len(lis)-2][tup2]
                ratio_lis.append(ratio)
            lis=lis[1:]
        sz=len(ratio_lis)
        final_ratio=0
        for ratio in ratio_lis:
            final_ratio+=ratio*(1/sz)
        sum+=math.log2(ratio)
    return sum*(-1/len(words))

In [122]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [123]:
#comparison over generated sentences
s='the door and turned the key fumbling in their panic'
n_val=5
print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with interpolation is',log_perp5(n_val,s))

Log2-perplexity without smoothing is 1.9812421762636143
Log2-perplexity with interpolation is 9.524394374722691


## PERFORMANCE ON TRAINING SET

In [218]:
text=""
filename='Harry_Potter_Text/'+'Book7.txt'
with open(filename) as f:
    Lines = f.readlines()
    # book=""
    for lin in Lines:
        if not (len(lin)>5 and lin[:6]=='Page |'):
            if lin != 'Harry Potter and the Goblet of Fire - J.K. Rowling \n':
                text+=lin.lower()
x=sent_tokenize(text)
test_sentences=[]
test_sentencewise_tokens=[]
for l in x:
    li=''.join(l.splitlines())
    # if not li[-1].isalnum():
    #     li=li[:-1]
    # li= '<s> '+li+' <e>'
    test_sentences.append(li)
    test_sentencewise_tokens.append(word_tokenize(li))
test_tokens=word_tokenize(text)





In [219]:
test_final_tokens=[]
for token in test_tokens:
    if not (len(token)==1 and not token.isalnum()):
        test_final_tokens.append(token.strip('.'))

test_tokens=test_final_tokens
#REMOVE PUNCTUATION


In [220]:
len(test_tokens)

204766

In [227]:
test_final_tokens=[]
for token in test_tokens:
    if token in tokens:
        test_final_tokens.append(token)

test_tokens=test_final_tokens
#REMOVE PUNCTUATION

In [228]:
len(test_tokens)

185632

In [229]:
test_book=' '.join(test_tokens)

In [230]:
n_val=5
print('Log2-perplexity by add-1 is',log_perp1(n_val,test_book))
print('Log2-perplexity by GT is',log_perp2(n_val,test_book))
print('Log2-perplexity by backoff is',log_perp3(n_val,test_book))
print('Log2-perplexity by KN is',log_perp4(n_val,test_book))
print('Log2-perplexity by interpolation is',log_perp5(n_val,test_book))

Log2-perplexity by add-1 is 19.623598669029754
Log2-perplexity by GT is 0.7538775399073951
Log2-perplexity by backoff is 46.44231070423704
Log2-perplexity by KN is 8.906824640255648
Log2-perplexity by interpolation is 5.533226856638685
